In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:

from tools import find_article_pubmed, get_article_pdf, query_vector_db_articles
# title = "Targeted therapy in advanced non-small cell lung cancer: current advances and future trends."

article = find_article_pubmed(article_title2)

Found a similar title with a similarity score of 100:
Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors.


In [94]:
get_article_pdf(article)

Title: Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors
Authors: Evangelos Terpos, Flora Zagouri, Michalis Liontos, Aimilia D. Sklirou, Konstantinos Koutsoukos, Christos Markellos, Alexandros Briasoulis, Eleni-Dimitra Papanagnou, Ioannis P. Trougakos, Meletios-Athanasios Dimopoulos
Journal: Journal of Hematology &amp; Oncology
Published Date: 2021-05-31
Open access PDF available at: https://jhoonline.biomedcentral.com/counter/pdf/10.1186/s13045-021-01099-x
Downloaded the PDF as: .saved_pdfs/10.1186_s13045-021-01099-x.pdf


'.saved_pdfs/10.1186_s13045-021-01099-x.pdf'

In [104]:
# pip install pinecone-client[grpc]
import os
from pinecone.grpc import PineconeGRPC as Pinecone

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("pubmed")

index.delete(delete_all=True)

In [3]:
article_title = "Targeted therapy in advanced non-small cell lung cancer: current advances and future trends."
query = "What disease are they discussing in the paper?"
query2 = "Can you list the protein targets they highlight in the paper?"
query3= "Summarize the main takeaways from the paper."

In [127]:
from langchain_google_vertexai import VertexAI
from langchain.chains import RetrievalQA  
# completion llm  
from langchain_cohere import ChatCohere

llm = ChatCohere(model="command-r-plus",temperature=0.0,max_tokens=4096)


from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [144]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

# from langchain.tools.retriever import create_retriever_tool

# vectorstore_search = create_retriever_tool(
#    retriever=retriever,
#    name="vectorstore_search",
#    description="Retrieve relevant info from a vectorstore that contains information from the article. You need to filter with the following metadata when you query the vector database: 'source': 'name_of_the_article' "
# )

# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource.
You are equipped with a special vectorstore of information about the scientific article.
"""

# Prompt template
# prompt = ChatPromptTemplate.from_template("{input}")
# Create the ReAct agent
agent = create_cohere_react_agent(
   llm=llm,
   tools=[query_vector_db_articles],
   prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent,
                               tools=[query_vector_db_articles],
                               verbose=True,
                               handle_parsing_errors=True,)


In [131]:
article_title2 = "Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors."

In [132]:
response = agent_executor.invoke({
   "input": "Summarize the main findings of the paper '{}'?".format(article_title2),
   "preamble": preamble,
})



> Entering new AgentExecutor chain...

I will search for the paper 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors' and summarise its main findings.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors', 'metadata': {}}}
[Document(page_content='Terpos\xa0 et\xa0al. J Hematol Oncol           (2021) 14:86  \nhttps://doi.org/10.1186/s13045-021-01099-x\nLETTER TO THE EDITOR\nLow titers of\xa0SARS -CoV -2 neutralizing \nantibodies after\xa0first vaccination dose in\xa0cancer \npatients receiving checkpoint inhibitors\nEvangelos Terpos1* , Flora Zagouri1, Michalis Liontos1, Aimilia D. Sklirou2, Konstantinos Koutsoukos1, \nChristos Markellos1, Alexandros Briasoulis1, Eleni‑Dimitra Papanagnou2, Ioannis P . Trougakos2 and \nMeletios‑Athanasios Dimopoulos1 \nAbstract \nVac

In [133]:
print(response['output'])


The paper found that cancer patients receiving checkpoint inhibitors had lower levels of SARS-CoV-2 neutralizing antibodies after the first vaccination dose compared to matched healthy volunteers. The study included 59 patients and 283 controls, and found that only 6 patients (10.7%) had NAb titers of equal to or more than 50%. The authors suggest that these results should be taken into consideration for the management of patients under treatment.


In [145]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

memory = ChatMessageHistory(session_id="test-session")



agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [146]:
agent_with_chat_history.invoke(
    {"input": "Summarize the main findings of the paper '{}'?".format(article_title2)},
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run 8e44c5d6-2784-4b94-8290-a2aaa94e2707 not found for run aaf40802-4dd0-481d-98e7-4483e02537de. Treating as a root run.




> Entering new AgentExecutor chain...

I will search for the paper 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors.' and summarise its main findings.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors.', 'metadata': {'title': 'article_title'}}}
No results found, Looking for the article online
No similar articles found on PubMed.
None
I could not find the paper. I will now search for the title of the paper to see if I can find it that way.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors.', 'metadata': {}}}
[Document(page_content='Terpos\xa0 et\xa0al. J Hematol Oncol           (2021) 14:86  \nhttps://doi.org/10.1186/s

{'input': "Summarize the main findings of the paper 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors.'?",
 'chat_history': [],
 'output': 'The paper found that the levels of SARS-CoV-2 neutralizing antibodies were significantly lower in cancer patients receiving checkpoint inhibitors compared to matched healthy volunteers up to day 22 post the first dose. The study also found that there were no new safety issues noticed. The authors suggest that these results should be taken into consideration for patients under treatment.',
 'citations': [CohereCitation(start=25, end=94, text='levels of SARS-CoV-2 neutralizing antibodies were significantly lower', documents=[{'output': Document(page_content='Terpos\xa0 et\xa0al. J Hematol Oncol           (2021) 14:86  \nhttps://doi.org/10.1186/s13045-021-01099-x\nLETTER TO THE EDITOR\nLow titers of\xa0SARS -CoV -2 neutralizing \nantibodies after\xa0first vaccination dose 

In [147]:
agent_with_chat_history.invoke(
    {"input": "What disease are they talking about?"},
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run 2305b5eb-f6ec-4d9d-9086-a37d76815153 not found for run b102e5f8-7bc5-41d2-a967-273c5223b5e9. Treating as a root run.




> Entering new AgentExecutor chain...

I will search for the paper and then write an answer based on the information I find.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors.', 'metadata': {'title': 'article_title'}}}
No results found, Looking for the article online
No similar articles found on PubMed.
NoneRelevant Documents: None
Cited Documents: None
Answer: I'm sorry, I couldn't find the paper. Can I help you with anything else?
Grounded answer: I'm sorry, I couldn't find the paper. Can I help you with anything else?

> Finished chain.


{'input': 'What disease are they talking about?',
 'chat_history': [HumanMessage(content="Summarize the main findings of the paper 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors.'?"),
  AIMessage(content='The paper found that the levels of SARS-CoV-2 neutralizing antibodies were significantly lower in cancer patients receiving checkpoint inhibitors compared to matched healthy volunteers up to day 22 post the first dose. The study also found that there were no new safety issues noticed. The authors suggest that these results should be taken into consideration for patients under treatment.')],
 'output': "I'm sorry, I couldn't find the paper. Can I help you with anything else?",
 'citations': []}

In [148]:
agent_with_chat_history.invoke(
    {"input": "What was my last question?"},
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run 8fa2d3ac-d9a0-445d-9705-28193bac2386 not found for run 735c0ed8-028d-4ecf-b240-79d30e06163f. Treating as a root run.




> Entering new AgentExecutor chain...

I will answer the question directly.
{'tool_name': 'directly_answer', 'parameters': {}}
directly_answer is not a valid tool, try one of [query_vector_db_articles].Relevant Documents: None
Cited Documents: None
Answer: Your last question was: "What disease are they talking about?"
Grounded answer: Your last question was: "What disease are they talking about?"

> Finished chain.


{'input': 'What was my last question?',
 'chat_history': [HumanMessage(content="Summarize the main findings of the paper 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors.'?"),
  AIMessage(content='The paper found that the levels of SARS-CoV-2 neutralizing antibodies were significantly lower in cancer patients receiving checkpoint inhibitors compared to matched healthy volunteers up to day 22 post the first dose. The study also found that there were no new safety issues noticed. The authors suggest that these results should be taken into consideration for patients under treatment.'),
  HumanMessage(content='What disease are they talking about?'),
  AIMessage(content="I'm sorry, I couldn't find the paper. Can I help you with anything else?")],
 'output': 'Your last question was: "What disease are they talking about?"',
 'citations': []}